In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [107]:
import numpy as np
import pandas as pd
import nltk
import preprocessing as prep
import multiprocessing
import random
import re

In [3]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from gensim.test.utils import get_tmpfile

2019-11-03 22:51:55,328 : INFO : 'pattern' package not found; tag filters are not available for English
2019-11-03 22:51:55,344 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-11-03 22:51:55,345 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [4]:
def df_tokenize(df, remove_stop_words):
    df = df.apply(lambda x: re.sub(r"http\S+|www.\S+", "", str(x))) #remove url
    df = df.apply(lambda x: re.sub(r"@\S+", "", x)) #remove mentions
    df = df.apply(lambda x: re.sub(r"[^\w\s@+]|\+", "", x)) #remove punc
    df = df.apply(lambda x: re.sub(r"\d+", "#", x)) #remove numbers
    df = df.apply(lambda x: re.sub(r"\s+", " ", x)) # remove multiple spaces
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL) 
    df = df.apply(lambda x: pattern.sub(r"\1\1", x)) #remove repeatitive letters in 2 words most like heeeey -> heey
    df = df.apply(lambda x: x.lower()) # GetLow!
    if remove_stop_words == True:
        from nltk.corpus import stopwords
        stop = stopwords.words("english")
        df = df.apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
    return df.apply(nltk.word_tokenize)

In [10]:
def get_fav_tweets(user):
    df = pd.read_csv(user+"/"+user+"_folls_favs_extented3.csv")
    return df_tokenize(df["text"], False)

In [6]:
def get_user_tweets(user):
    df = pd.read_csv(user+"/"+user+"_tweets.csv")
    return df_tokenize(df["text"], False)

In [11]:
favs = get_fav_tweets("TheTweetOfGod")

In [18]:
user_tweets = get_user_tweets("TheTweetOfGod")

In [19]:
test_corpus = []
for i in range(user_tweets.shape[0]):
    test_corpus.append(TaggedDocument(user_tweets[i], [i]))

In [15]:
train_corpus = []
for i in range(favs.shape[0]):
    train_corpus.append(TaggedDocument(favs[i], [i]))

In [105]:
workers = multiprocessing.cpu_count()

In [ ]:
model = 

In [143]:
model = Doc2Vec(vector_size=200, dm=0, min_count=3, epochs=50, workers=workers)

In [144]:
model.build_vocab(train_corpus)

2019-11-03 16:43:52,278 : INFO : collecting all words and their counts
2019-11-03 16:43:52,279 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-11-03 16:43:52,358 : INFO : PROGRESS: at example #10000, processed 183197 words (2358679/s), 21856 word types, 10000 tags
2019-11-03 16:43:52,418 : INFO : PROGRESS: at example #20000, processed 357389 words (2921758/s), 34116 word types, 20000 tags
2019-11-03 16:43:52,500 : INFO : PROGRESS: at example #30000, processed 527346 words (2091531/s), 42655 word types, 30000 tags
2019-11-03 16:43:52,618 : INFO : PROGRESS: at example #40000, processed 718587 words (1637978/s), 51439 word types, 40000 tags
2019-11-03 16:43:52,687 : INFO : PROGRESS: at example #50000, processed 900211 words (2632010/s), 59344 word types, 50000 tags
2019-11-03 16:43:52,749 : INFO : PROGRESS: at example #60000, processed 1074262 words (2867146/s), 66138 word types, 60000 tags
2019-11-03 16:43:52,819 : INFO : PROGRESS: at example #70000, 

In [147]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2019-11-03 16:44:56,915 : INFO : training model with 4 workers on 33322 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2019-11-03 16:44:57,982 : INFO : EPOCH 1 - PROGRESS: at 13.84% examples, 294059 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:44:59,008 : INFO : EPOCH 1 - PROGRESS: at 28.41% examples, 305651 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:45:00,019 : INFO : EPOCH 1 - PROGRESS: at 42.52% examples, 308187 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:45:01,030 : INFO : EPOCH 1 - PROGRESS: at 56.52% examples, 314843 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:45:02,041 : INFO : EPOCH 1 - PROGRESS: at 70.16% examples, 314490 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:45:03,064 : INFO : EPOCH 1 - PROGRESS: at 84.09% examples, 316148 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:45:04,093 : INFO : EPOCH 1 - PROGRESS: at 96.88% examples, 312659 words/s, in_qsize 8, out_qsize 0
2019-11-03 16:45:04,281 : INFO : worker thread finished; aw

2019-11-03 16:45:50,138 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-03 16:45:50,151 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-03 16:45:50,155 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-03 16:45:50,156 : INFO : EPOCH - 6 : training on 2843520 raw words (2310543 effective words) took 8.0s, 287839 effective words/s
2019-11-03 16:45:51,180 : INFO : EPOCH 7 - PROGRESS: at 12.07% examples, 263890 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:45:52,225 : INFO : EPOCH 7 - PROGRESS: at 25.44% examples, 276491 words/s, in_qsize 8, out_qsize 0
2019-11-03 16:45:53,244 : INFO : EPOCH 7 - PROGRESS: at 38.63% examples, 280191 words/s, in_qsize 8, out_qsize 0
2019-11-03 16:45:54,258 : INFO : EPOCH 7 - PROGRESS: at 51.51% examples, 285755 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:45:55,313 : INFO : EPOCH 7 - PROGRESS: at 64.42% examples, 287104 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:45:56,353 :

2019-11-03 16:46:39,348 : INFO : EPOCH 13 - PROGRESS: at 24.68% examples, 275065 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:46:40,378 : INFO : EPOCH 13 - PROGRESS: at 37.20% examples, 272904 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:46:41,391 : INFO : EPOCH 13 - PROGRESS: at 48.90% examples, 272368 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:46:42,408 : INFO : EPOCH 13 - PROGRESS: at 60.18% examples, 272173 words/s, in_qsize 8, out_qsize 0
2019-11-03 16:46:43,416 : INFO : EPOCH 13 - PROGRESS: at 68.77% examples, 259052 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:46:44,421 : INFO : EPOCH 13 - PROGRESS: at 77.47% examples, 250880 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:46:45,427 : INFO : EPOCH 13 - PROGRESS: at 90.02% examples, 256629 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:46:46,075 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-03 16:46:46,088 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-03 16:46:46,

2019-11-03 16:47:30,234 : INFO : EPOCH 19 - PROGRESS: at 53.50% examples, 239096 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:47:31,246 : INFO : EPOCH 19 - PROGRESS: at 64.41% examples, 242008 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:47:32,267 : INFO : EPOCH 19 - PROGRESS: at 78.15% examples, 251761 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:47:33,320 : INFO : EPOCH 19 - PROGRESS: at 91.67% examples, 258872 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:47:34,011 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-03 16:47:34,029 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-03 16:47:34,048 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-03 16:47:34,049 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-03 16:47:34,050 : INFO : EPOCH - 19 : training on 2843520 raw words (2310510 effective words) took 8.9s, 259329 effective words/s
2019-11-03 16:47:35,072 : INFO : EPOCH 20 

2019-11-03 16:48:20,533 : INFO : EPOCH 25 - PROGRESS: at 48.23% examples, 265921 words/s, in_qsize 8, out_qsize 0
2019-11-03 16:48:21,572 : INFO : EPOCH 25 - PROGRESS: at 58.52% examples, 261073 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:48:22,584 : INFO : EPOCH 25 - PROGRESS: at 66.64% examples, 248439 words/s, in_qsize 8, out_qsize 0
2019-11-03 16:48:23,590 : INFO : EPOCH 25 - PROGRESS: at 75.76% examples, 243008 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:48:24,591 : INFO : EPOCH 25 - PROGRESS: at 87.68% examples, 248779 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:48:25,609 : INFO : EPOCH 25 - PROGRESS: at 97.79% examples, 246819 words/s, in_qsize 6, out_qsize 0
2019-11-03 16:48:25,734 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-03 16:48:25,776 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-03 16:48:25,796 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-03 16:48:25,830 : INFO : worker thr

2019-11-03 16:49:10,619 : INFO : EPOCH 31 - PROGRESS: at 57.07% examples, 320623 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:49:11,683 : INFO : EPOCH 31 - PROGRESS: at 71.60% examples, 319093 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:49:12,685 : INFO : EPOCH 31 - PROGRESS: at 81.78% examples, 306589 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:49:13,706 : INFO : EPOCH 31 - PROGRESS: at 92.10% examples, 296793 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:49:14,399 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-03 16:49:14,436 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-03 16:49:14,453 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-03 16:49:14,459 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-03 16:49:14,460 : INFO : EPOCH - 31 : training on 2843520 raw words (2310464 effective words) took 7.9s, 291872 effective words/s
2019-11-03 16:49:15,494 : INFO : EPOCH 32 

2019-11-03 16:50:00,588 : INFO : EPOCH 37 - PROGRESS: at 37.93% examples, 272828 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:50:01,599 : INFO : EPOCH 37 - PROGRESS: at 50.51% examples, 278319 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:50:02,648 : INFO : EPOCH 37 - PROGRESS: at 62.85% examples, 279912 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:50:03,656 : INFO : EPOCH 37 - PROGRESS: at 75.47% examples, 280339 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:50:04,670 : INFO : EPOCH 37 - PROGRESS: at 88.00% examples, 283390 words/s, in_qsize 8, out_qsize 0
2019-11-03 16:50:05,560 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-03 16:50:05,570 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-03 16:50:05,596 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-03 16:50:05,598 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-03 16:50:05,598 : INFO : EPOCH - 37 : training on 2843520 

2019-11-03 16:50:50,524 : INFO : EPOCH 43 - PROGRESS: at 82.74% examples, 309976 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:50:51,576 : INFO : EPOCH 43 - PROGRESS: at 95.80% examples, 307456 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:50:51,825 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-03 16:50:51,838 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-03 16:50:51,857 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-03 16:50:51,860 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-03 16:50:51,861 : INFO : EPOCH - 43 : training on 2843520 raw words (2310573 effective words) took 7.5s, 308536 effective words/s
2019-11-03 16:50:52,922 : INFO : EPOCH 44 - PROGRESS: at 12.39% examples, 262805 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:50:53,967 : INFO : EPOCH 44 - PROGRESS: at 25.44% examples, 271811 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:50:54,972 : INFO : EPOCH 44 

2019-11-03 16:51:38,835 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-03 16:51:38,838 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-03 16:51:38,838 : INFO : EPOCH - 49 : training on 2843520 raw words (2309972 effective words) took 7.4s, 310291 effective words/s
2019-11-03 16:51:39,888 : INFO : EPOCH 50 - PROGRESS: at 13.84% examples, 297073 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:51:40,901 : INFO : EPOCH 50 - PROGRESS: at 28.09% examples, 305186 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:51:41,906 : INFO : EPOCH 50 - PROGRESS: at 41.41% examples, 303283 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:51:42,920 : INFO : EPOCH 50 - PROGRESS: at 55.17% examples, 309102 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:51:43,949 : INFO : EPOCH 50 - PROGRESS: at 68.83% examples, 308865 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:51:44,969 : INFO : EPOCH 50 - PROGRESS: at 82.74% examples, 311441 words/s, in_qsize 8, out_qsiz

2019-11-03 16:52:26,009 : INFO : EPOCH - 55 : training on 2843520 raw words (2310904 effective words) took 9.0s, 256158 effective words/s
2019-11-03 16:52:27,028 : INFO : EPOCH 56 - PROGRESS: at 11.20% examples, 250370 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:52:28,045 : INFO : EPOCH 56 - PROGRESS: at 24.07% examples, 265708 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:52:29,057 : INFO : EPOCH 56 - PROGRESS: at 36.00% examples, 265547 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:52:30,065 : INFO : EPOCH 56 - PROGRESS: at 48.52% examples, 271275 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:52:31,082 : INFO : EPOCH 56 - PROGRESS: at 60.79% examples, 276149 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:52:32,082 : INFO : EPOCH 56 - PROGRESS: at 72.66% examples, 274719 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:52:33,092 : INFO : EPOCH 56 - PROGRESS: at 85.02% examples, 277754 words/s, in_qsize 7, out_qsize 0
2019-11-03 16:52:34,114 : INFO : EPOCH 56 - PROGRESS: at 97.79% 

In [150]:
fname = get_tmpfile("model_tweet_200")
model.save(fname)

2019-11-03 17:21:34,578 : INFO : saving Doc2Vec object under C:\Users\BEKIRU~1\AppData\Local\Temp\model_tweet_200, separately None
2019-11-03 17:21:34,580 : INFO : storing np array 'vectors_docs' to C:\Users\BEKIRU~1\AppData\Local\Temp\model_tweet_200.docvecs.vectors_docs.npy
2019-11-03 17:21:35,734 : INFO : saved C:\Users\BEKIRU~1\AppData\Local\Temp\model_tweet_200


In [13]:
model = Doc2Vec.load("model_tweet_200/model_tweet_200")

2019-11-03 23:34:37,771 : INFO : loading Doc2Vec object from model_tweet_200/model_tweet_200
2019-11-03 23:34:38,464 : INFO : loading vocabulary recursively from model_tweet_200/model_tweet_200.vocabulary.* with mmap=None
2019-11-03 23:34:38,464 : INFO : loading trainables recursively from model_tweet_200/model_tweet_200.trainables.* with mmap=None
2019-11-03 23:34:38,464 : INFO : loading wv recursively from model_tweet_200/model_tweet_200.wv.* with mmap=None
2019-11-03 23:34:38,464 : INFO : loading docvecs recursively from model_tweet_200/model_tweet_200.docvecs.* with mmap=None
2019-11-03 23:34:38,474 : INFO : loading vectors_docs from model_tweet_200/model_tweet_200.docvecs.vectors_docs.npy with mmap=None
2019-11-03 23:34:38,875 : INFO : loaded model_tweet_200/model_tweet_200


In [175]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [104]:
X = model.docvecs.similarity(1200,108164)
X

0.5471989

In [83]:
inferred_vector = model.infer_vector(train_corpus[1200].words)
model.docvecs.most_similar([inferred_vector], topn=10)

[(1200, 0.9480443596839905),
 (55365, 0.6151461601257324),
 (89256, 0.5866603851318359),
 (13499, 0.5860309600830078),
 (2754, 0.5839618444442749),
 (53541, 0.575958251953125),
 (148156, 0.5728983879089355),
 (45429, 0.5721272230148315),
 (103455, 0.5637922286987305),
 (108164, 0.5551384091377258)]

In [169]:
test_vectors = np.array([model.infer_vector(test_corpus[i].words) for i in range(len(test_corpus))])

In [173]:
sims1 = [model.docvecs.most_similar([test_vectors[i]], topn=len(train_corpus)) for i in range(0,500)]
for x in range(0,500):
    df = pd.DataFrame(sims1[x])
    df.to_csv("simsData_new/sims_"+str(x).zfill(4)+".csv", index=False)
del df
del sims1

In [174]:
sims2 = [model.docvecs.most_similar([test_vectors[i]], topn=len(train_corpus)) for i in range(500,1000)]
n = 500
for x in range(0,500):
    df = pd.DataFrame(sims2[x])
    df.to_csv("simsData_new/sims_"+str(n).zfill(4)+".csv", index=False)
    n += 1
del df
del sims2

In [175]:
sims3 = [model.docvecs.most_similar([test_vectors[i]], topn=len(train_corpus)) for i in range(1000,1500)]
n = 1000
for x in range(0,500):
    df = pd.DataFrame(sims3[x])
    df.to_csv("simsData_new/sims_"+str(n).zfill(4)+".csv", index=False)
    n += 1
del df
del sims3

In [176]:
sims4 = [model.docvecs.most_similar([test_vectors[i]], topn=len(train_corpus)) for i in range(1500,2000)]
n = 1500
for x in range(0,500):
    df = pd.DataFrame(sims4[x])
    df.to_csv("simsData_new/sims_"+str(n).zfill(4)+".csv", index=False)
    n += 1
del df
del sims4

In [177]:
sims5 = [model.docvecs.most_similar([test_vectors[i]], topn=len(train_corpus)) for i in range(2000,len(test_corpus))]
n = 2000
for x in range(0,len(test_corpus)-2000):
    df = pd.DataFrame(sims5[x])
    df.to_csv("simsData_new/sims_"+str(n).zfill(4)+".csv", index=False)
    n += 1
del df
del sims5